In [1]:
# from arcgis.gis import GIS
import requests
import pandas as pd
import altair as alt

## Retrieve JSON data from ArcGIS REST API

URL is available on the Vital Signs Open Data Portal - API Explorer

In [2]:
url = "https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Salepr/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"
r = requests.get(url)
json = r.json()
json.keys()

# json['features'][0]

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

### Process Data

Transform the data from wide to long to fit for the data visualization

In [3]:
df = pd.json_normalize(json['features'])
df.head(2)

,attributes.OBJECTID,attributes.CSA2010,attributes.salepr10,attributes.salepr11,attributes.salepr12,attributes.salepr13,attributes.salepr14,attributes.salepr15,attributes.salepr16,attributes.salepr17,attributes.salepr18,attributes.salepr19,attributes.Shape__Area,attributes.Shape__Length,geometry.rings
0,1,Allendale/Irvington/S. Hilton,42500,30000,47500,31805.5,33250,35500,46639.5,55000,75000.0,77250.0,6.377046e+07,38770.165571,"[[[-76.6572574296439, 39.2760020837069], [-76...."
1,2,Beechfield/Ten Hills/West Hills,142250,120000,144501,139500.0,130000,126000,118450.0,139950,159450.0,149000.0,4.788253e+07,37524.950533,"[[[-76.6947913455734, 39.3020124459612], [-76...."


In [4]:
df_long = pd.wide_to_long(df, stubnames='attributes.salepr', i=['attributes.CSA2010'], j = 'year').reset_index()
df_long = df_long[['attributes.CSA2010', 'attributes.salepr', 'year']]

df_long.columns = ['Entity', 'Price', 'Year']
df_long.head(2)

,Entity,Price,Year
0,Allendale/Irvington/S. Hilton,42500.0,10
1,Beechfield/Ten Hills/West Hills,142250.0,10


## Example Visualizations

In [5]:
alt.Chart(df_long).mark_area().encode(
    x=alt.X('Year:O', axis=alt.Axis(labels=True)),
    y=alt.Y('Price:Q'),
    color=alt.Color('Entity', legend=None),
    row=alt.Row('Entity',
        header=alt.Header(labelAngle=0,labelAlign='left')
    )
).properties(height=50, width=400, title='Median Price of Homes Sold')


alt.Chart(...)

In [6]:
alt.Chart(df_long).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('Year:O', axis=alt.Axis(labelAngle=0)),
    alt.Y('Entity:N'),
    alt.Size('Price:Q',
        scale=alt.Scale(range=[0, 400]),
        legend=alt.Legend(title='Median Price of Homes Sold')
    ),
    alt.Color('Entity:N', legend=None)
).properties(
    width=450,
    height=1000
)

alt.Chart(...)